In [19]:
import enum

import isort
import sys
import platform
import importlib
from types import ModuleType

In [20]:
# module_names = isort.stdlibs.py38.stdlib

In [21]:
def get_stdlib_packages():
    if sys.version_info.minor == 10:
        module_names = sys.stdlib_module_names
    else:
        module_names = isort.stdlibs.py38.stdlib

    external_packages = set()
    for name in module_names:
        if name[0] == '_' or name == 'this' or name == 'antigravity':
            continue
        external_packages.add(name)
    return external_packages

In [22]:
def task1():
    external_packages = get_stdlib_packages()
    v = sys.version_info
    print(f'Python {v.major}.{v.minor}.{v.micro} on {platform.system()} {platform.release()}')
    print(f'Stdlib contains {len(external_packages)} external modules and packages: ')
    print(external_packages)

In [23]:
task1()

Python 3.10.6 on Linux 5.15.0-48-generic
Stdlib contains 215 external modules and packages: 
{'winreg', 're', 'ntpath', 'glob', 'argparse', 'types', 'codecs', 'mailcap', 'textwrap', 'smtpd', 'resource', 'dbm', 'pickletools', 'shelve', 'netrc', 'modulefinder', 'posix', 'syslog', 'lib2to3', 'enum', 'pydoc_data', 'wsgiref', 'shlex', 'sunau', 'audioop', 'cmd', 'ast', 'optparse', 'filecmp', 'configparser', 'random', 'decimal', 'fractions', 'runpy', 'selectors', 'string', 'imaplib', 'profile', 'gettext', 'xdrlib', 'xml', 'pty', 'binascii', 'nis', 'turtle', 'graphlib', 'builtins', 'sndhdr', 'itertools', 'faulthandler', 'pwd', 'dataclasses', 'difflib', 'pyclbr', 'uuid', 'tracemalloc', 'site', 'sre_constants', 'importlib', 'unicodedata', 'uu', 'sqlite3', 'codeop', 'math', 'bz2', 'queue', 'pprint', 'contextlib', 'copy', 'token', 'nt', 'reprlib', 'urllib', 'chunk', 'csv', 'tempfile', 'pdb', 'ensurepip', 'email', 'array', 'bisect', 'venv', 'tkinter', 'sre_compile', 'tokenize', 'socket', 'fcntl', '

In [24]:
def get_real_packages(package_names):
    real_modules = set()
    not_importable_modules = set()
    for name in package_names:
        try:
            importlib.import_module(name)
            real_modules.add(name)
            # if name != 'importlib':
            #     del sys.modules[name]
        except:
            not_importable_modules.add(name)
    return real_modules, not_importable_modules

In [25]:
def get_real():
    external_packages = get_stdlib_packages()
    return get_real_packages(external_packages)

In [26]:
def task2():
    v = sys.version_info
    real_modules, not_importable_modules = get_real()
    print(f'These StdLib packages on Python-{v.major}.{v.minor}.{v.micro}/{platform.system()} {platform.release()} '
          f'are not importable:')
    print(not_importable_modules)

    # for name in real_modules:
    #     del sys.modules[name]

In [27]:
task2()

These StdLib packages on Python-3.10.6/Linux 5.15.0-48-generic are not importable:
{'nt', 'turtle', 'msilib', 'tkinter', 'winreg', 'winsound', 'idlelib', 'msvcrt', 'turtledemo', 'ensurepip'}


In [28]:
def module_dependency(module_names, name):
    if name not in module_names:
        raise Exception(f'{name} is not importable module')
    dp_names = set()

    try:
        importlib.import_module(name)
    except:
        print(f'err name: {name}')
    for key, val in vars(sys.modules[name]).items():
        if isinstance(val, ModuleType):
            md_name = val.__name__

            try:
                index = md_name.index(".")
                md_name = md_name[0:index]
            except:
                pass

            dp_names.add(md_name)
            # print(f'key: {key}, type: {type(val)}, val: {md_name}, module: {md_name}')

    # del sys.modules[name]

    return dp_names

In [29]:
def core_modules(real_modules):
    # real_modules, _ = get_real()
    core_module_names = set()
    for r_name in real_modules:
        dp_names = module_dependency(real_modules, r_name)
        if len(dp_names) == 0:
            core_module_names.add(r_name)

    return core_module_names


In [30]:
def most_dependent_modules(real_modules):
    dp_dict = dict((name, 0) for name in real_modules)
    for md_name in real_modules:
        count = len(module_dependency(real_modules, md_name))
        dp_dict[md_name] = count

    sorted_dp_dict = dict(sorted(dp_dict.items(), key=lambda item: item[1], reverse=True))
    i = 0
    most_dependent_module_names = dict()
    for k, v in sorted_dp_dict.items():
        if i > 5:
            break
        most_dependent_module_names[k] = v
        i = i + 1

    return most_dependent_module_names



In [31]:
def task3():
    real_modules, not_importable_modules = get_real()

    # re-import libs:
    for r in real_modules:
        pass

    # print(f'real modules: {real_modules}')
    core_module_names = core_modules(real_modules)

    dp_dict = most_dependent_modules(real_modules)

    print(f'The following StdLib packages are most dependent:')
    for k, v in dp_dict.items():
        print(f'{k}: {v}')
    print(f'The {len(core_module_names)} core packages are:')
    print(core_module_names)

    # for name in real_modules:
    #     del sys.modules[name]

In [32]:
task3()

The following StdLib packages are most dependent:
zipfile: 17
inspect: 17
pdb: 15
subprocess: 15
pydoc: 15
logging: 12
The 59 core packages are:
{'ossaudiodev', 'numbers', 'difflib', 'readline', 'quopri', 'termios', 'marshal', 'nturl2path', 'types', 'imghdr', 'abc', 'stat', 'sre_constants', 'resource', 'heapq', 'unicodedata', 'contextvars', 'posix', 'syslog', 'math', 'gc', 'spwd', 'functools', 'wsgiref', 'copy', 'token', 'sunau', 'audioop', 'operator', 'zlib', 'atexit', 'stringprep', 'tty', 'colorsys', 'errno', 'chunk', 'decimal', 'struct', 'select', 'time', 'sre_parse', 'binascii', 'nis', 'array', 'bisect', 'graphlib', 'builtins', 'sndhdr', 'itertools', 'mmap', 'keyword', 'faulthandler', 'cmath', 'fcntl', 'opcode', 'sys', 'pwd', 'copyreg', 'grp'}


In [33]:
def explore_package(package_name):
    importlib.import_module(package_name)
    pack = sys.modules[package_name]
    file_name = 'builtin_binary'
    try:
        file_name = pack.__file__
        n = len(file_name)
        # print(file_name[n-3:n])
        if file_name[n-3:n] == '.py':
            # print(file_name)
            return package_name, file_name, 0
        elif file_name[n-3:n] == '.so':
            # print(file_name)
            return package_name, file_name, 1
        else:
            raise Exception(f'unsupported file: {file_name}')
    except:
        return package_name, file_name, 2

p, f, r = explore_package('locale')
print(r)

0


In [43]:
def get_py_modules(module_names):
    py_mds = set()
    for md in module_names:
        p, f, t = explore_package(md)
        if t == 0:
            py_mds.add((p, f))
    return py_mds


In [47]:
real_modules, _ = get_real()

py_mds = get_py_modules(real_modules)

# print(py_mds)

In [45]:
def count_file_line(file_path):
    count=0
    for line in open(file_path, encoding="utf8"):
        count+=1
    return count


num_lines = count_file_line('/usr/lib/python3.10/calendar.py')
print(num_lines)


759


In [46]:
def get_file_lines_dict(module_names):
    md_lines = dict()
    py_mds = get_py_modules(module_names)
    for py_md in py_mds:
        key = py_md[0]
        num_lines = count_file_line(py_md[1])
        md_lines[key] = num_lines
    return md_lines


md_lines = get_file_lines_dict(real_modules)
print(md_lines)


{'queue': 326, 'shutil': 1509, 'keyword': 63, 'tempfile': 1014, 'configparser': 1367, 'pydoc_data': 0, 'string': 280, 'pyclbr': 314, 'importlib': 176, 'gettext': 803, 'subprocess': 2104, 'pipes': 247, 'zoneinfo': 31, 'csv': 444, 'optparse': 1681, 'cProfile': 190, 'plistlib': 902, 'ipaddress': 2287, 'socketserver': 844, 'contextvars': 4, 'json': 359, 'platform': 1312, 'zipfile': 2474, 'ftplib': 981, 'tarfile': 2614, 'cgi': 1004, 'selectors': 619, 'lib2to3': 8, 'fileinput': 462, 'decimal': 11, 'curses': 101, 'random': 930, 'doctest': 2810, 'abc': 188, 'sunau': 526, 'sre_compile': 808, 'contextlib': 745, 'copyreg': 219, 'symtable': 322, 'stat': 195, 'profile': 610, 'xmlrpc': 1, 'cmd': 401, 'sysconfig': 845, 're': 383, 'graphlib': 246, 'argparse': 2595, 'os': 1122, 'enum': 1053, 'sqlite3': 71, 'asynchat': 315, 'traceback': 692, 'runpy': 321, 'tty': 36, 'imghdr': 168, 'functools': 992, 'signal': 92, 'mailbox': 2151, 'fnmatch': 199, 'aifc': 947, 'reprlib': 161, 'dbm': 189, 'nturl2path': 81, 

In [35]:
import inspect
import fileinput
import socket
# from types import
import enum

for name, obj in inspect.getmembers(socket):
        if inspect.isclass(obj):
            if issubclass(obj, enum.Enum):
                print(f'{obj}, enum')
            else:
                print(f'{obj}, type: {type(obj).__class__}')

<enum 'AddressFamily'>, enum
<enum 'AddressInfo'>, enum
<enum 'IntEnum'>, enum
<enum 'IntFlag'>, enum
<enum 'MsgFlag'>, enum
<class 'socket.SocketIO'>, type: <class 'type'>
<enum 'SocketKind'>, enum
<class '_socket.socket'>, type: <class 'type'>
<class 'socket._GiveupOnSendfile'>, type: <class 'type'>
<class 'OSError'>, type: <class 'type'>
<class 'socket.gaierror'>, type: <class 'type'>
<class 'socket.herror'>, type: <class 'type'>
<class 'socket.socket'>, type: <class 'type'>
<class 'TimeoutError'>, type: <class 'type'>


In [36]:
real_modules, _ = get_real()

md_class_count = dict()

for md in real_modules:
    count = 0
    # cl = set()
    for name, obj in inspect.getmembers(sys.modules[md]):
        if inspect.isclass(obj):
            if issubclass(obj, enum.Enum):
                # print(f'{obj}, enum')
                continue
            else:
                # print(f'{obj}, type: {type(obj).__class__}')
                count = count + 1
                # cl.add(obj.__name__)
    md_class_count[md] = count

sorted_cl_dict = dict(sorted(md_class_count.items(), key=lambda item: item[1], reverse=True))
i = 0
largest_pack_cl = dict()
for k, v in sorted_cl_dict.items():
    if i > 5:
        break
    largest_pack_cl[k] = v
    i = i + 1

print(largest_pack_cl)




{'ast': 136, 'builtins': 94, 'typing': 50, 'asyncio': 47, 'ctypes': 43, 'types': 28}


In [ ]:
import locale

# from locale import localeconv as 

https://stackoverflow.com/questions/34511673/extracting-comments-from-python-source-code

use this for task 5